<a href="https://colab.research.google.com/github/aliasgharchakera/CUDA_Spring_2024/blob/main/Midterm/ay06993_midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-a3ldc2pt
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-a3ldc2pt
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Source files will be saved in "/tmp/tmpvg3pxrcb".


In [16]:
%%cuda
#include <stdio.h>
#define MAX_ITERATIONS 100

int main(){
  float a[1024][1024];
  float l[1024][1024];
  for(int k=0;k<MAX_ITERATIONS-1;k++)
  {
    for(int j=1;j<1023;j++)
    {
        for(int i=1;i<1023;i++)
        {
          l[i][j] = (a[i+1][j] + a[i-1][j] + a[i][j+1] + a[i][j-1]) / 4;
        }
    }
    for(int j=1;j<1023;j++)
    {
        for(int i=1;i<1023;i++)
        {
          a[i][j] = l[i][j];
        }
    }
  }
  printf("Hello World");
  return 0;
}

In [31]:
%%cuda
#include <stdio.h>
#include <stdlib.h>

#define MAX_ITERATIONS 100
#define N 1024

__global__ void kernel(float* a, float* l)
{
   int i = blockIdx.x * blockDim.x + threadIdx.x;
   int j = blockIdx.y * blockDim.y + threadIdx.y;

   if (i > 0 && i < N-1 && j > 0 && j < N-1)
   {
      l[i * N + j] = (a[(i+1) * N + j] + a[(i-1) * N + j] + a[i * N + (j+1)] + a[i * N + (j-1)]) / 4;
   }
}

int main()
{
    const int SIZE = N*N;
    float *d_a, *d_l;

    // Allocate memory for host arrays
    float* h_a = (float*)malloc(SIZE * sizeof(float));
    float* h_l = (float*)malloc(SIZE * sizeof(float));
    for (int i = 0; i < SIZE; i++)
    {
        h_a[i] = 1.0;
        h_l[i] = 1.0;
    }

    // Allocate memory on the device
    cudaMalloc((void**)&d_a, SIZE * sizeof(float));
    cudaMalloc((void**)&d_l, SIZE * sizeof(float));

    // Copy data from host to device
    cudaMemcpy(d_a, h_a, SIZE * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_l, h_l, SIZE * sizeof(float), cudaMemcpyHostToDevice);

    dim3 blockDim(16, 16);
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x, (N + blockDim.y - 1) / blockDim.y);

    for (int k = 0; k < MAX_ITERATIONS - 1; k++)
    {
        kernel<<<gridDim, blockDim>>>(d_a, d_l);
        cudaDeviceSynchronize();
        cudaMemcpy(d_a, d_l, SIZE * sizeof(float), cudaMemcpyDeviceToDevice);
    }

    // Copy data from device to host
    cudaMemcpy(h_a, d_a, SIZE * sizeof(float), cudaMemcpyDeviceToHost);

    printf("a[0][0] = %f\n", h_a[0]);
    printf("a[N-1][N-1] = %f\n", h_a[SIZE-1]);

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_l);

    // Free memory for host arrays
    free(h_a);
    free(h_l);

    return 0;
}

a[0][0] = 1.000000
a[N-1][N-1] = 1.000000

